In [ ]:
from scipy import misc
import numpy as np
from __future__ import division
from sklearn import metrics
from itertools import cycle

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from matplotlib._png import read_png
from matplotlib.cbook import get_sample_data
from matplotlib.lines import Line2D

%matplotlib inline
np.set_printoptions(threshold='nan')

In [ ]:
def BinaryClassifier(T, img, T_idx):
    height, width = img.shape
    binImg = np.zeros_like(img)
    
    totalNumberOfPixels = height*width
    
    if T_idx==1: # NSS
        numberOfSalientPixels = np.flatnonzero(img).size # The ones that are not black
    else:
        numberOfSalientPixels = totalNumberOfPixels
    
    TSalienPixels = (T * numberOfSalientPixels)/100
    
    # Convert it into a 1D array
    img_1d = img.flatten()
    
    # Find the indices in the 1D array
    idx_1d = img_1d.argsort()[-TSalienPixels:]

    # convert the idx_1d back into indices arrays for each dimension
    x_idx, y_idx = np.unravel_index(idx_1d, img.shape)


    if T_idx==1: # NSS
        for x, y, in zip(x_idx, y_idx):
            binImg[x][y] = 1
    else:
        for x, y, in zip(x_idx, y_idx):
            binImg[x][y] = 255
    
    
    return binImg


In [ ]:
def ROC(setT, binImgG, imgP):
    #**************************************************************
    # setT -- Set of Thresholds to be used with imgP
    # binImgG -- binary clasiffied Ground-truth image
    # imgP -- Image to be binary classified using setT 
    #**************************************************************

    index = 0
    TruePositiveRate = np.zeros(len(setT), dtype=float)
    FalsePositiveRate = np.zeros(len(setT), dtype=float)
    N = np.flatnonzero(binImgG).size # Number of fixations

    for t in setT:
        binImgP = BinaryClassifier(t, imgP, 2)
        TP, FN, FP, TN = 0, 0, 0, 0
        
        for i in range(binImgP.shape[0]):
            for j in range(binImgP.shape[1]):
                if binImgG[i][j] == 255:
                    if binImgP[i][j] == 255:
                        TP += 1
                    else:
                        FN += 1
                else:
                    if binImgP[i][j] == 255:
                        FP += 1
                    else:
                        TN += 1
#        print TP, FP, FN, TN
        
        TruePositiveRate[index] = TP/(TP + FN)
        FalsePositiveRate[index] = FP/(FP + TN)
        
 #       TruePositiveRate[index] = TP/N
 #       FalsePositiveRate[index] = FP/N
        index += 1
        
        ''' print TP, FP, FN, TN
        
        plt.figure()  
        plt.imshow(binImgG, cmap="gray") 
        plt.figure()  
        plt.imshow(binImgP, cmap="gray") '''
        

    # Plot ROC curve
    
    AUC = metrics.auc(FalsePositiveRate, TruePositiveRate) 
   
    
    # Plot ROC curve 
    '''    lw = 2
    plt.plot(FalsePositiveRate, TruePositiveRate, color='darkorange', lw=lw, label='ROC curve (area = %0.2f)' %  AUC)
    
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()'''
 
    
    return AUC, FalsePositiveRate, TruePositiveRate
            
            
            
            

In [ ]:
def NSS (FixG, imgP):

    # Binarize FixG
    # numberOfSalientPixels = np.flatnonzero(FixG).size !!! Modify in BinaryClassifier
    BFixG = BinaryClassifier(100, FixG, 1)
    
    print 
    
    imgP = misc.bytescale(imgP, high=1, low=0)
    # Standardized saliency Map
    Z = (imgP - np.mean(imgP))/np.std(imgP) 
    N = np.flatnonzero(BFixG).size # Number of fixations
    
    nss = 1/N * np.sum(Z*BFixG)
    return nss
    

In [ ]:
imgG = misc.imread('C:\Users\Ana\Code\webvr-samples-master\panorama-viewer-master\Saliency\10s\bwGroupSaliency_The_Porch.jpg', mode = 'L') # Ground Truth
FixG = misc.imread('C:\Users\Ana\Code\webvr-samples-master\panorama-viewer-master\Saliency\10s\GroupFix_The_Porch_0.jpg.jpg', mode = 'L') # Ground Truth

#names = ['aliconDemo_BikeTeam.png', 'aliency_BikeTeam.jpg', 'aliency_translated_BikeTeam_3.jpg', 'DeepGaze2Demo_BikeTeam1.png', 'DeepGaze2Demo_BikeTeam2.png']

names = ['mergedSaliency_The_Porch.jpg', 'The_Porch_merged3.jpg']
setT= np.linspace(0.01, 100, num=20) # For the ROC method

fpr = np.zeros((len(names), len(setT)))
tpr = np.zeros((len(names), len(setT)))
AUC = np.zeros(len(names))
nss = np.zeros(len(names))

for i in range(len(names)):
    filename = 'C:\Users\Ana\Code\Images\QoMex\Salicon\\'  +  names[i]

    imgP = misc.imread(filename, mode = 'L') # Predicted saliency map


    # Resizing the images - So, they have the same size
    if imgG.shape[0] < imgP.shape [0]:
        imgP = misc.imresize(imgP, imgG.shape)
    else:
        imgG = misc.imresize(imgG, imgP.shape)
        

    # Binarization of imgG according to the threshold T
    T = 20 # In percentage
    binImgG = BinaryClassifier(T, imgG, 2)

    # Call the ROC method (Judd?)
    AUC[i], fpr[i], tpr[i] = ROC(setT, binImgG, imgP)
    
    # Call the S-ROC method (shuffled AUC)
    #S_AUC[i], S_fpr[i], S_tpr[i] = ROC(setT, binImgG, imgP)
    
    # Call the NSS method 
    # Resizing the images - So, they have the same size
    if FixG.shape[0] < imgP.shape [0]:
        imgP = misc.imresize(imgP, imgG.shape)
    else:
        FixG = misc.imresize(imgG, imgP.shape)
        
    nss[i] = NSS(FixG, imgP)




In [ ]:
# Correlation-based metric to compare two saliency maps


img1 = misc.imread('C:\Users\Ana\Code\webvr-samples-master\panorama-viewer-master\Saliency\Results_20s_DBSCAN\GroupSaliencyMap_BikeTeam.jpg', mode = 'L') 
img2 = misc.imread('C:\Users\Ana\Code\webvr-samples-master\panorama-viewer-master\Saliency\Results_10s_DBSCAN\GroupSaliencyMap_BikeTeam.jpg', mode = 'L') 

a = np.ravel(img1)
b = np.ravel(img2)
c = np.column_stack((a,b))

res = np.cov(c)/(np.std(img1.flatten())*np.std(img2.flatten()))
print res



In [ ]:
# ROC plot - From values got from matlab code

names = ['Salicon & FSM', 'Salicon', 'Judd & FSM', 'Judd', 'Judd-E & FSM', 'Judd-E', 'DG2 & FSM', 'DG2' ]

fpr = np.zeros((len(names), 102))
tpr = np.zeros((len(names), 102))
AUC = np.zeros(len(names))


# Plot ROC curve     
colors = cycle(['steelblue', 'steelblue', 'indianred', 'indianred','olivedrab','olivedrab', 'darkorange', 'darkorange', 'magenta', 'magenta', 'cornflowerblue', 'magenta', 'salmon'])  
fig=plt.figure()


# 0: Salicon Merged solution
fpr[0] =  [0.0000,0.0003,0.0005,0.0007,0.0011,0.0012,0.0015,0.0017,0.0021,0.0024,0.0027,0.0030,0.0033,0.0035,0.0038,0.0042,0.0044,0.0048,0.0052,0.0055,0.0060,0.0065,0.0071,0.0076,0.0081,0.0085,0.0088,0.0095,0.0101,0.0104,0.0110,0.0116,0.0123,0.0131,0.0139,0.0148,0.0157,0.0164,0.0172,0.0180,0.0190,0.0200,0.0211,0.0222,0.0235,0.0246,0.0259,0.0272,0.0286,0.0302,0.0318,0.0332,0.0349,0.0367,0.0386,0.0406,0.0431,0.0452,0.0473,0.0496,0.0521,0.0549,0.0575,0.0605,0.0639,0.0675,0.0709,0.0741,0.0774,0.0816,0.0850,0.0886,0.0929,0.0976,0.1021,0.1068,0.1122,0.1176,0.1235,0.1294,0.1360,0.1424,0.1496,0.1575,0.1670,0.1774,0.1876,0.1992,0.2131,0.2264,0.2407,0.2584,0.2779,0.3003,0.3266,0.3830,0.4414,0.4775,0.6517,0.7682,0.9720,1.0000]
tpr[0] = [0.0000,0.0008,0.0031,0.0057,0.0081,0.0107,0.0135,0.0162,0.0190,0.0212,0.0234,0.0253,0.0269,0.0287,0.0307,0.0329,0.0355,0.0384,0.0410,0.0436,0.0465,0.0512,0.0559,0.0600,0.0638,0.0677,0.0720,0.0765,0.0807,0.0847,0.0893,0.0946,0.0988,0.1035,0.1078,0.1113,0.1148,0.1185,0.1222,0.1260,0.1302,0.1335,0.1382,0.1427,0.1473,0.1524,0.1568,0.1629,0.1683,0.1731,0.1797,0.1866,0.1927,0.1989,0.2056,0.2119,0.2185,0.2242,0.2313,0.2393,0.2462,0.2553,0.2642,0.2752,0.2848,0.2980,0.3103,0.3220,0.3350,0.3481,0.3592,0.3705,0.3818,0.3943,0.4051,0.4149,0.4253,0.4349,0.4457,0.4602,0.4737,0.4870,0.5001,0.5125,0.5263,0.5429,0.5622,0.5815,0.6016,0.6208,0.6383,0.6553,0.6730,0.6968,0.7196,0.7479,0.7870,0.8075,0.8750,0.9111,0.9884,1.0000]

# 1: Salicon Original solution
fpr[1] =  [0.0000,0.0003,0.0004,0.0006,0.0007,0.0009,0.0011,0.0013,0.0014,0.0017,0.0020,0.0023,0.0027,0.0029,0.0032,0.0036,0.0042,0.0046,0.0051,0.0054,0.0059,0.0063,0.0068,0.0073,0.0076,0.0083,0.0086,0.0089,0.0096,0.0102,0.0108,0.0114,0.0119,0.0126,0.0133,0.0140,0.0149,0.0157,0.0164,0.0173,0.0184,0.0194,0.0204,0.0212,0.0221,0.0230,0.0240,0.0247,0.0257,0.0266,0.0277,0.0291,0.0305,0.0315,0.0328,0.0343,0.0358,0.0373,0.0391,0.0408,0.0427,0.0445,0.0466,0.0485,0.0509,0.0531,0.0553,0.0577,0.0599,0.0628,0.0655,0.0683,0.0716,0.0747,0.0777,0.0812,0.0849,0.0892,0.0933,0.0978,0.1027,0.1079,0.1133,0.1191,0.1259,0.1332,0.1419,0.1513,0.1617,0.1719,0.1845,0.1994,0.2149,0.2329,0.2546,0.2836,0.3185,0.4240,0.5192,0.6889,1.0000,1.0000]
tpr[1] = [0.0000,0.0013,0.0038,0.0063,0.0078,0.0097,0.0116,0.0135,0.0151,0.0169,0.0194,0.0218,0.0244,0.0262,0.0281,0.0308,0.0334,0.0367,0.0395,0.0426,0.0456,0.0485,0.0516,0.0551,0.0591,0.0630,0.0671,0.0709,0.0746,0.0792,0.0833,0.0873,0.0918,0.0953,0.0987,0.1014,0.1047,0.1080,0.1117,0.1149,0.1181,0.1214,0.1251,0.1286,0.1316,0.1349,0.1379,0.1411,0.1450,0.1491,0.1541,0.1604,0.1658,0.1713,0.1760,0.1802,0.1852,0.1910,0.1966,0.2023,0.2071,0.2133,0.2197,0.2257,0.2330,0.2383,0.2447,0.2525,0.2601,0.2682,0.2758,0.2834,0.2908,0.2980,0.3054,0.3131,0.3208,0.3305,0.3388,0.3489,0.3610,0.3702,0.3790,0.3875,0.3971,0.4044,0.4158,0.4292,0.4425,0.4574,0.4721,0.4853,0.4976,0.5137,0.5294,0.5545,0.5754,0.6467,0.7097,0.8036,1.0000,1.0000]

# 2: Judd Merged solution
fpr[2] =  [0.0000,0.0004,0.0007,0.0011,0.0014,0.0018,0.0023,0.0028,0.0035,0.0043,0.0050,0.0058,0.0067,0.0079,0.0091,0.0105,0.0120,0.0138,0.0154,0.0176,0.0199,0.0226,0.0250,0.0281,0.0312,0.0347,0.0382,0.0428,0.0479,0.0527,0.0574,0.0634,0.0704,0.0779,0.0848,0.0922,0.0999,0.1091,0.1186,0.1285,0.1385,0.1495,0.1600,0.1715,0.1830,0.1948,0.2074,0.2207,0.2340,0.2462,0.2599,0.2730,0.2865,0.3007,0.3152,0.3297,0.3439,0.3598,0.3760,0.3931,0.4098,0.4259,0.4430,0.4604,0.4790,0.4974,0.5144,0.5312,0.5486,0.5663,0.5833,0.6004,0.6171,0.6352,0.6530,0.6712,0.6905,0.7103,0.7290,0.7480,0.7666,0.7845,0.8043,0.8237,0.8425,0.8591,0.8755,0.8924,0.9104,0.9264,0.9420,0.9576,0.9697,0.9818,0.9907,0.9963,0.9987,0.9997,1.0000,1.0000,1.0000,1.0000]
tpr[2] = [0.0000,0.0003,0.0012,0.0023,0.0035,0.0044,0.0064,0.0087,0.0117,0.0144,0.0170,0.0204,0.0248,0.0307,0.0357,0.0399,0.0444,0.0498,0.0554,0.0612,0.0680,0.0763,0.0844,0.0928,0.1016,0.1116,0.1232,0.1375,0.1539,0.1707,0.1871,0.2019,0.2195,0.2343,0.2521,0.2704,0.2867,0.3030,0.3197,0.3395,0.3599,0.3779,0.3954,0.4103,0.4270,0.4455,0.4622,0.4790,0.4942,0.5088,0.5244,0.5402,0.5556,0.5716,0.5905,0.6064,0.6270,0.6455,0.6628,0.6787,0.6954,0.7092,0.7219,0.7341,0.7489,0.7637,0.7798,0.7958,0.8083,0.8210,0.8309,0.8397,0.8499,0.8614,0.8728,0.8818,0.8890,0.8959,0.9033,0.9112,0.9195,0.9276,0.9336,0.9382,0.9441,0.9497,0.9559,0.9617,0.9664,0.9710,0.9758,0.9808,0.9858,0.9907,0.9943,0.9972,0.9992,0.9999,1.0000,1.0000,1.0000,1.0000]

# 3: Judd Original solution
fpr[3] =  [0.0000,0.0003,0.0006,0.0010,0.0015,0.0022,0.0033,0.0043,0.0053,0.0064,0.0081,0.0097,0.0115,0.0130,0.0148,0.0172,0.0196,0.0220,0.0250,0.0283,0.0320,0.0357,0.0399,0.0446,0.0492,0.0546,0.0605,0.0669,0.0728,0.0802,0.0875,0.0953,0.1033,0.1125,0.1215,0.1311,0.1405,0.1506,0.1616,0.1729,0.1841,0.1957,0.2073,0.2201,0.2325,0.2450,0.2579,0.2718,0.2875,0.3040,0.3205,0.3368,0.3552,0.3738,0.3929,0.4117,0.4305,0.4498,0.4685,0.4876,0.5062,0.5246,0.5443,0.5642,0.5844,0.6034,0.6226,0.6420,0.6602,0.6787,0.6972,0.7156,0.7333,0.7516,0.7683,0.7862,0.8027,0.8199,0.8372,0.8540,0.8699,0.8849,0.8995,0.9121,0.9242,0.9349,0.9445,0.9543,0.9623,0.9707,0.9766,0.9820,0.9871,0.9910,0.9938,0.9963,0.9979,0.9988,0.9995,0.9999,1.0000,1.0000]
tpr[3] = [0.0000,0.0005,0.0025,0.0047,0.0067,0.0105,0.0153,0.0209,0.0266,0.0335,0.0406,0.0480,0.0544,0.0615,0.0686,0.0777,0.0868,0.0959,0.1043,0.1120,0.1205,0.1309,0.1447,0.1591,0.1722,0.1840,0.1945,0.2052,0.2176,0.2307,0.2424,0.2563,0.2730,0.2883,0.2999,0.3116,0.3234,0.3359,0.3470,0.3594,0.3702,0.3817,0.3952,0.4091,0.4201,0.4303,0.4402,0.4492,0.4605,0.4720,0.4818,0.4910,0.5001,0.5106,0.5228,0.5332,0.5468,0.5585,0.5691,0.5801,0.5910,0.6001,0.6091,0.6195,0.6286,0.6385,0.6484,0.6593,0.6697,0.6823,0.6927,0.7050,0.7151,0.7232,0.7335,0.7428,0.7546,0.7665,0.7774,0.7858,0.7944,0.8043,0.8161,0.8305,0.8444,0.8583,0.8744,0.8873,0.8996,0.9116,0.9244,0.9393,0.9494,0.9596,0.9676,0.9761,0.9826,0.9874,0.9927,0.9973,0.9993,1.0000]



# 4: Judd_Horizon Merged solution
fpr[4] =  [0.0000,0.0005,0.0009,0.0013,0.0019,0.0027,0.0038,0.0047,0.0060,0.0076,0.0094,0.0116,0.0138,0.0165,0.0202,0.0243,0.0293,0.0349,0.0410,0.0479,0.0544,0.0613,0.0680,0.0761,0.0850,0.0945,0.1036,0.1131,0.1233,0.1335,0.1452,0.1563,0.1679,0.1786,0.1903,0.2020,0.2137,0.2268,0.2393,0.2504,0.2623,0.2740,0.2848,0.2959,0.3074,0.3191,0.3307,0.3428,0.3550,0.3674,0.3787,0.3905,0.4016,0.4139,0.4264,0.4381,0.4509,0.4636,0.4749,0.4860,0.4978,0.5100,0.5215,0.5339,0.5468,0.5598,0.5713,0.5850,0.5978,0.6110,0.6244,0.6370,0.6492,0.6632,0.6775,0.6905,0.7037,0.7174,0.7307,0.7441,0.7576,0.7715,0.7864,0.8007,0.8159,0.8317,0.8470,0.8636,0.8782,0.8938,0.9080,0.9230,0.9373,0.9501,0.9634,0.9748,0.9852,0.9927,0.9972,0.9999,1.0000,1.0000]
tpr[4] = [0.0000,0.0006,0.0021,0.0048,0.0092,0.0133,0.0181,0.0232,0.0297,0.0374,0.0451,0.0543,0.0638,0.0765,0.0916,0.1090,0.1290,0.1512,0.1762,0.2035,0.2278,0.2496,0.2731,0.2964,0.3184,0.3404,0.3615,0.3851,0.4058,0.4281,0.4484,0.4696,0.4914,0.5072,0.5247,0.5400,0.5531,0.5667,0.5816,0.5975,0.6106,0.6244,0.6375,0.6500,0.6620,0.6753,0.6865,0.6967,0.7063,0.7158,0.7236,0.7326,0.7400,0.7489,0.7575,0.7660,0.7737,0.7822,0.7876,0.7926,0.7977,0.8027,0.8080,0.8127,0.8182,0.8240,0.8297,0.8358,0.8432,0.8496,0.8540,0.8594,0.8650,0.8714,0.8768,0.8823,0.8892,0.8948,0.9004,0.9067,0.9134,0.9183,0.9242,0.9307,0.9373,0.9437,0.9498,0.9549,0.9607,0.9658,0.9703,0.9746,0.9785,0.9834,0.9889,0.9932,0.9966,0.9985,0.9994,1.0000,1.0000,1.0000]

# 5: Judd_Horizon Original solution
fpr[5] =  [0.0000,0.0003,0.0008,0.0013,0.0018,0.0026,0.0034,0.0047,0.0059,0.0073,0.0088,0.0105,0.0133,0.0159,0.0188,0.0227,0.0268,0.0318,0.0376,0.0443,0.0508,0.0574,0.0644,0.0720,0.0793,0.0871,0.0963,0.1050,0.1141,0.1237,0.1338,0.1453,0.1564,0.1676,0.1797,0.1915,0.2034,0.2152,0.2262,0.2383,0.2490,0.2607,0.2711,0.2833,0.2940,0.3050,0.3166,0.3291,0.3406,0.3529,0.3649,0.3772,0.3897,0.4017,0.4133,0.4262,0.4394,0.4526,0.4649,0.4775,0.4892,0.5017,0.5138,0.5268,0.5403,0.5534,0.5668,0.5807,0.5941,0.6076,0.6203,0.6333,0.6459,0.6589,0.6722,0.6860,0.6997,0.7139,0.7279,0.7411,0.7554,0.7701,0.7853,0.8008,0.8165,0.8321,0.8468,0.8626,0.8789,0.8952,0.9106,0.9251,0.9382,0.9504,0.9623,0.9740,0.9849,0.9920,0.9971,0.9996,1.0000,1.0000]
tpr[5] = [0.0000,0.0015,0.0031,0.0066,0.0097,0.0128,0.0163,0.0209,0.0269,0.0352,0.0450,0.0553,0.0663,0.0762,0.0888,0.1021,0.1189,0.1404,0.1617,0.1832,0.2058,0.2282,0.2488,0.2699,0.2908,0.3113,0.3325,0.3548,0.3763,0.3955,0.4153,0.4382,0.4600,0.4798,0.4974,0.5121,0.5270,0.5424,0.5557,0.5665,0.5777,0.5910,0.6028,0.6132,0.6244,0.6354,0.6454,0.6557,0.6680,0.6808,0.6917,0.7024,0.7112,0.7196,0.7288,0.7399,0.7486,0.7582,0.7677,0.7773,0.7838,0.7906,0.7958,0.8006,0.8057,0.8114,0.8172,0.8233,0.8303,0.8370,0.8440,0.8505,0.8571,0.8648,0.8709,0.8770,0.8829,0.8893,0.8963,0.9028,0.9077,0.9142,0.9212,0.9271,0.9329,0.9398,0.9465,0.9517,0.9574,0.9620,0.9675,0.9723,0.9777,0.9829,0.9876,0.9908,0.9948,0.9971,0.9990,0.9999,1.0000,1.0000]


# # 6: DG2_MIT Merged solution
# fpr[6] =  [0.0000,0.0003,0.0005,0.0005,0.0006,0.0008,0.0010,0.0013,0.0014,0.0016,0.0016,0.0018,0.0021,0.0022,0.0023,0.0026,0.0028,0.0031,0.0034,0.0036,0.0039,0.0042,0.0044,0.0045,0.0048,0.0050,0.0055,0.0059,0.0063,0.0066,0.0069,0.0072,0.0075,0.0079,0.0082,0.0086,0.0090,0.0095,0.0100,0.0103,0.0108,0.0111,0.0118,0.0124,0.0128,0.0134,0.0140,0.0148,0.0155,0.0164,0.0173,0.0180,0.0188,0.0194,0.0201,0.0211,0.0222,0.0232,0.0242,0.0253,0.0267,0.0280,0.0294,0.0308,0.0322,0.0337,0.0350,0.0364,0.0378,0.0394,0.0409,0.0436,0.0456,0.0479,0.0507,0.0532,0.0561,0.0593,0.0630,0.0666,0.0708,0.0746,0.0794,0.0846,0.0902,0.0963,0.1034,0.1475,0.1547,0.1630,0.1720,0.1834,0.2283,0.2414,0.2561,0.2743,0.3855,0.4389,0.6321,0.8913,1.0000,1.0000]
# tpr[6] = [0.0000,0.0006,0.0020,0.0041,0.0055,0.0069,0.0083,0.0102,0.0121,0.0138,0.0154,0.0171,0.0188,0.0206,0.0227,0.0249,0.0272,0.0295,0.0317,0.0342,0.0369,0.0393,0.0419,0.0450,0.0472,0.0497,0.0520,0.0549,0.0574,0.0595,0.0615,0.0639,0.0659,0.0677,0.0695,0.0716,0.0733,0.0756,0.0775,0.0796,0.0823,0.0850,0.0875,0.0904,0.0932,0.0964,0.0999,0.1026,0.1056,0.1091,0.1127,0.1167,0.1207,0.1242,0.1286,0.1329,0.1375,0.1427,0.1472,0.1522,0.1570,0.1625,0.1674,0.1732,0.1796,0.1861,0.1916,0.1977,0.2041,0.2103,0.2176,0.2252,0.2323,0.2407,0.2520,0.2597,0.2670,0.2776,0.2900,0.3006,0.3096,0.3191,0.3286,0.3406,0.3510,0.3637,0.3756,0.3981,0.4157,0.4333,0.4490,0.4707,0.5160,0.5359,0.5603,0.5823,0.6477,0.6853,0.7968,0.9340,1.0000,1.0000]

# # 7: DG2_MIT Original solution
# fpr[7] =  [0.0000,0.0004,0.0005,0.0006,0.0008,0.0009,0.0011,0.0012,0.0013,0.0013,0.0015,0.0016,0.0018,0.0019,0.0021,0.0022,0.0025,0.0027,0.0029,0.0031,0.0033,0.0036,0.0037,0.0040,0.0043,0.0046,0.0048,0.0051,0.0054,0.0058,0.0062,0.0065,0.0068,0.0073,0.0076,0.0080,0.0084,0.0088,0.0092,0.0096,0.0101,0.0106,0.0112,0.0116,0.0122,0.0125,0.0131,0.0137,0.0142,0.0151,0.0157,0.0165,0.0174,0.0181,0.0188,0.0196,0.0205,0.0216,0.0226,0.0233,0.0241,0.0250,0.0260,0.0272,0.0285,0.0296,0.0310,0.0326,0.0342,0.0360,0.0377,0.0393,0.0415,0.0434,0.0457,0.0480,0.0504,0.0533,0.0560,0.0593,0.0628,0.0663,0.0705,0.0747,0.0797,0.0853,0.0918,0.0985,0.1061,0.1147,0.1239,0.1693,0.1793,0.1922,0.2421,0.2577,0.3804,0.4651,0.5512,0.6364,0.9639,1.0000]
# tpr[7] = [0.0000,0.0015,0.0036,0.0054,0.0074,0.0097,0.0114,0.0136,0.0158,0.0173,0.0196,0.0217,0.0235,0.0249,0.0267,0.0292,0.0319,0.0351,0.0374,0.0394,0.0414,0.0433,0.0450,0.0465,0.0485,0.0503,0.0525,0.0546,0.0567,0.0590,0.0619,0.0643,0.0664,0.0691,0.0712,0.0736,0.0762,0.0788,0.0806,0.0826,0.0841,0.0862,0.0884,0.0909,0.0930,0.0954,0.0976,0.1006,0.1033,0.1059,0.1090,0.1125,0.1163,0.1196,0.1233,0.1267,0.1305,0.1348,0.1401,0.1441,0.1483,0.1535,0.1584,0.1626,0.1675,0.1733,0.1798,0.1849,0.1910,0.1967,0.2040,0.2099,0.2152,0.2207,0.2268,0.2333,0.2402,0.2474,0.2531,0.2602,0.2667,0.2744,0.2824,0.2926,0.3039,0.3190,0.3336,0.3467,0.3637,0.3810,0.3977,0.4422,0.4623,0.4802,0.5310,0.5526,0.6369,0.6837,0.7411,0.7921,0.9863,1.0000]

# 8: DG2_AUC Merged solution
fpr[6] =  [0.0000,0.0087,0.0164,0.0263,0.0341,0.0433,0.0521,0.0608,0.0705,0.0799,0.0895,0.0989,0.1092,0.1185,0.1289,0.1398,0.1517,0.1627,0.1740,0.1838,0.1947,0.2054,0.2157,0.2263,0.2366,0.2477,0.2577,0.2670,0.2764,0.2867,0.2966,0.3064,0.3163,0.3264,0.3365,0.3470,0.3566,0.3679,0.3784,0.3892,0.4003,0.4102,0.4206,0.4305,0.4415,0.4516,0.4625,0.4728,0.4829,0.4924,0.5021,0.5128,0.5227,0.5330,0.5422,0.5524,0.5626,0.5723,0.5821,0.5929,0.6027,0.6129,0.6229,0.6329,0.6426,0.6527,0.6627,0.6724,0.6817,0.6914,0.7011,0.7106,0.7201,0.7296,0.7388,0.7484,0.7590,0.7684,0.7781,0.7869,0.7974,0.8076,0.8171,0.8279,0.8376,0.8484,0.8579,0.8692,0.8789,0.8896,0.8994,0.9094,0.9200,0.9304,0.9402,0.9494,0.9607,0.9692,0.9793,0.9870,0.9997,1.0000]
tpr[6] = [0.0000,0.0641,0.0998,0.1508,0.1855,0.2178,0.2427,0.2633,0.2889,0.3121,0.3347,0.3606,0.3817,0.4029,0.4226,0.4392,0.4591,0.4771,0.4945,0.5101,0.5265,0.5408,0.5531,0.5635,0.5753,0.5881,0.6003,0.6123,0.6214,0.6336,0.6439,0.6526,0.6614,0.6725,0.6817,0.6905,0.6992,0.7104,0.7219,0.7312,0.7396,0.7464,0.7530,0.7591,0.7648,0.7708,0.7773,0.7830,0.7895,0.7954,0.8006,0.8055,0.8110,0.8166,0.8224,0.8281,0.8344,0.8410,0.8461,0.8510,0.8556,0.8597,0.8637,0.8689,0.8759,0.8813,0.8863,0.8908,0.8952,0.8983,0.9016,0.9047,0.9090,0.9122,0.9156,0.9202,0.9247,0.9280,0.9317,0.9350,0.9392,0.9434,0.9480,0.9536,0.9576,0.9607,0.9627,0.9665,0.9685,0.9709,0.9730,0.9759,0.9784,0.9817,0.9861,0.9901,0.9929,0.9946,0.9976,0.9983,1.0000,1.0000]

# 9: DG2_AUC Original solution
fpr[7] =  [0.0000,0.0115,0.0206,0.0311,0.0419,0.0505,0.0612,0.0697,0.0802,0.0887,0.0992,0.1083,0.1194,0.1307,0.1395,0.1501,0.1593,0.1700,0.1792,0.1895,0.1992,0.2094,0.2188,0.2285,0.2394,0.2484,0.2594,0.2690,0.2789,0.2887,0.2996,0.3099,0.3197,0.3300,0.3396,0.3501,0.3597,0.3698,0.3791,0.3892,0.3988,0.4094,0.4196,0.4287,0.4386,0.4481,0.4579,0.4676,0.4782,0.4872,0.4970,0.5067,0.5166,0.5268,0.5360,0.5462,0.5555,0.5666,0.5763,0.5867,0.5975,0.6069,0.6181,0.6273,0.6378,0.6479,0.6590,0.6687,0.6790,0.6889,0.6986,0.7095,0.7186,0.7294,0.7380,0.7488,0.7582,0.7687,0.7788,0.7886,0.7997,0.8088,0.8196,0.8284,0.8389,0.8475,0.8585,0.8674,0.8775,0.8884,0.8978,0.9091,0.9180,0.9296,0.9380,0.9492,0.9582,0.9690,0.9793,0.9879,1.0000,1.0000]
tpr[7] = [0.0000,0.0902,0.1331,0.1796,0.2158,0.2427,0.2674,0.2886,0.3128,0.3321,0.3575,0.3759,0.3964,0.4081,0.4221,0.4407,0.4539,0.4663,0.4783,0.4936,0.5051,0.5155,0.5262,0.5356,0.5478,0.5575,0.5681,0.5763,0.5859,0.5940,0.6029,0.6104,0.6177,0.6245,0.6316,0.6410,0.6517,0.6611,0.6678,0.6747,0.6832,0.6926,0.6994,0.7043,0.7093,0.7155,0.7243,0.7299,0.7377,0.7458,0.7520,0.7585,0.7649,0.7739,0.7809,0.7882,0.7952,0.8044,0.8114,0.8208,0.8298,0.8371,0.8462,0.8524,0.8584,0.8623,0.8680,0.8721,0.8770,0.8821,0.8858,0.8932,0.8986,0.9083,0.9142,0.9199,0.9246,0.9304,0.9359,0.9419,0.9460,0.9493,0.9543,0.9570,0.9602,0.9639,0.9687,0.9711,0.9748,0.9776,0.9793,0.9820,0.9840,0.9869,0.9882,0.9894,0.9909,0.9939,0.9957,0.9969,1.0000,1.0000]


AUC =[0.7409, 0.6697, 0.6998, 0.5640, 0.7336, 0.7252, 0.6989, 0.6745,  0.7318,  0.7094]

markers = np.array([">",">","o", "o", "s", "s", "D", "D", "x", "x", "d", "p"])
plt.style.use('seaborn-whitegrid') 

for i, color in zip(range(len(names)), colors):
    if i== 1 or i==3 or i==5 or i==7 or i==9:
        if i==1:
            plt.plot(fpr[i], tpr[i], '--', color=color, lw=2, marker = markers[i],markevery=1, label='{0} (AUC = {1:0.3f})' ''.format(names[i], AUC[i]))
        else:
            plt.plot(fpr[i], tpr[i], '--', color=color, lw=2, marker = markers[i],markevery=10, label='{0} (AUC = {1:0.3f})' ''.format(names[i], AUC[i]))
    else:
        plt.plot(fpr[i], tpr[i], color=color, lw=2, marker = markers[i],markevery=8, label='{0} (AUC = {1:0.3f})' ''.format(names[i], AUC[i]))
    #plt.plot(setT, tpr[i], color=color, lw=2, label='ROC curve {0} (area = {1:0.2f})' ''.format(i, AUC[i]))

plt.plot([0, 1], [0, 1], 'k:', lw=2)
plt.xlim([0.0, 1.0])
#plt.xlim([0.0, 30])
plt.ylim([0.0, 1.00])

plt.xlabel('False Positive Rate',fontsize=12)
plt.ylabel('True Positive Rate',fontsize=12)
#plt.xlabel('Percent Salient')
#plt.title('Receiver operating characteristic (ROC) - Indoor3')
plt.legend(loc="lower right", bbox_to_anchor=(1.01, -0.01),fontsize=10)
fig.savefig('C:/Users/Ana/Code/webvr-samples-master/panorama-viewer-master/Saliency/ROC_saliencyModels_white.pdf')

plt.show()